In [8]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition

from langchain.chat_models import init_chat_model
model = init_chat_model("openai:gpt-4.1")

async with MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            "args": ["./math_server.py"],
            "transport": "stdio",
        },
        "transparencia": {
            "command": "python",
            "args": ["./mcp_server_transparencia.py"],
            "transport": "stdio",
        }
    }
) as client:
    tools = client.get_tools()
    def call_model(state: MessagesState):
        response = model.bind_tools(tools).invoke(state["messages"])
        return {"messages": response}

    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_node(ToolNode(tools))
    builder.add_edge(START, "call_model")
    builder.add_conditional_edges(
        "call_model",
        tools_condition,
    )
    builder.add_edge("tools", "call_model")
    graph = builder.compile()
    math_response = await graph.ainvoke({"messages": "what's (3 + 5) x 12?"})
    weather_response = await graph.ainvoke({"messages": "Cuantos contratos tiene el proveedor con ruc 20500000001?"})

In [13]:
math_response

{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='c127de5d-179d-40b3-822a-8853a7610ad9'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QLiWNeEbeaDwrkQ4Gqp1B1FB', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 162, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_a1102cf978', 'id': 'chatcmpl-BV7G7lGKSaA73xmRnfRiPZwDVFTsO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--95ad7584-85ec-4719-a94c-0abc17629861-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_QLiWNeEbeaDwrkQ4Gqp1B1FB', 'typ

In [14]:
weather_response

{'messages': [HumanMessage(content='Cuantos contratos tiene el proveedor con ruc 20500000001?', additional_kwargs={}, response_metadata={}, id='cdbf6482-6b90-4600-b4e6-a0a9212396fe'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NrqK0ZBGWiqi61msI40eL0nY', 'function': {'arguments': '{"ruc":"20500000001"}', 'name': 'buscar_contratos_por_ruc'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 165, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_a1102cf978', 'id': 'chatcmpl-BV7G9kIztS7UThV9AQQPoMqbikVMg', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e7ba89a3-4525-4cf4-86ce-ed0ebc83f966-0', tool_calls=[{'name': 'buscar_co

In [16]:
weather_response['messages']

[HumanMessage(content='Cuantos contratos tiene el proveedor con ruc 20500000001?', additional_kwargs={}, response_metadata={}, id='cdbf6482-6b90-4600-b4e6-a0a9212396fe'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NrqK0ZBGWiqi61msI40eL0nY', 'function': {'arguments': '{"ruc":"20500000001"}', 'name': 'buscar_contratos_por_ruc'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 165, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_a1102cf978', 'id': 'chatcmpl-BV7G9kIztS7UThV9AQQPoMqbikVMg', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e7ba89a3-4525-4cf4-86ce-ed0ebc83f966-0', tool_calls=[{'name': 'buscar_contratos_por_ru